Python script to scrape player in-game statistics data from fbref.com. Extracted data from 14 professional soccer leagues around the world.

In [1]:
# libraries
import pandas as pd
import numpy as np
import requests
import time
import random

years = [2023]
years_special = ['2022-2023']
numbers = [11,20,27,26,9,12,50,30,13,40,37,23,39,32]
stats = ['passing','defense','shooting']
leagues = ['-Serie-A-Stats','-Bundesliga-Stats','-Super-League-Greece-Stats','-Super-Lig-Stats',
           '-Premier-League-Stats','-La-Liga-Stats','-Superliga-Stats','-Russian-Premier-League-Stats',
           '-Ligue-1-Stats','-Scottish-Premiership-Stats','-Belgian-Pro-League-Stats','-Eredivisie-Stats',
           '-Ukrainian-Premier-League-Stats','-Primeira-Liga-Stats']
           
league_names = ['Italy (Serie A)','Germany (Bundesliga)','Greece (Super League)',
               ' Turkey (Super Lig)','England (Premier League)','Spain (La Liga)', 'Denmark (Superliga)', 
               'Russia (Premier Liga)', 'France (Ligue 1)', 'Scotland (Scottish Premiership)',
               'Belgium (Jupiler Pro League)', 'Netherlands (Eredivisie)', 'Ukraine (Premier Liga)',
               'Portugal (Primeira Liga)']
                
defense_list = []
passing_list = []
shooting_list = []
count = 0
for n in range(len(numbers)):
    if count < 3:
        for s in stats:
            for y in years:
                data = 'https://fbref.com/en/comps/' + str(numbers[count]) + '/' + str(y) + '/' + s + '/' + str(y) + leagues[count]
                response = requests.get(data).text.replace('<!--', '').replace('-->', '')
                df = pd.read_html(response, header=1)[2]
                df['Based'] = league_names[count]
                if s == 'passing':
                    passing_list.append(df)
                elif s == 'defense':
                    defense_list.append(df)
                else:
                    shooting_list.append(df)
                time.sleep(3)
                print(count)
    else:
        for t in stats:
            for z in years_special:
                data = 'https://fbref.com/en/comps/' + str(numbers[count]) + '/' + z + '/' + t + '/' + z + leagues[count]
                response = requests.get(data).text.replace('<!--', '').replace('-->', '')
                df = pd.read_html(response, header=1)[2]
                df['Based'] = league_names[count]
                if t == 'passing':
                    passing_list.append(df)
                elif t == 'defense':
                    defense_list.append(df)
                else:
                    shooting_list.append(df)
                time.sleep(3)
                print(count)

    count += 1

ValueError: No tables found

In [ ]:
shot = []
defense = []
passing = []
for i in range(len(years_special)):
    shot.append(shooting_list[i::len(years_special)])
    defense.append(defense_list[i::len(years_special)])
    passing.append(passing_list[i::len(years_special)])

In [ ]:
#for y in range(len(years_special)):
for k in range(3):
    if stats[k] == 'passing':
        count = 0
        for df in passing[0]:
            # rename columns
            df = df.fillna(0)

#             df = df.drop(columns=['Rk', 'Nation', 'Pos', 'A-xAG',
#                                  '1/3', 'PPA', 'CrsPA', 'Matches','TotDist', 'PrgDist'])

            df.rename(columns={'Cmp':'Total Cmp', 'Att': 'Total Att', 'Cmp%':'Total Cmp%','Cmp.1':'Short Cmp', 
                               'Att.1':'Short Att','Cmp%.1':'Short Cmp%', 'Cmp.2':'Medium Cmp', 'Att.2':'Medium Att',
                               'Cmp%.2':'Medium Cmp%', 'Cmp.3':'Long Cmp', 'Att.3':'Long Att', 'Cmp%.3':'Long Cmp%'}, inplace = True)
            passing[0][count] = df

            count += 1

    elif stats[k] == 'defense':
        count = 0
        for df in defense[0]:

            # rename columns
            df = df.fillna(0)

            df['Age'] = df['Age'].str[:2]
            df['Position_2'] = df['Pos'].str[3:]
            df['Position'] = df['Pos'].str[:2]
            df['Nation'] = df['Nation'].str.split(' ').str.get(1)

            df['Position'] = df['Position'].replace({'MF': 'Midfielder', 'DF': 'Defender', 'FW': 'Forward', 'GK': 'Goalkeeper'})
            df['Position_2'] = df['Position_2'].replace({'MF': 'Midfielder', 'DF': 'Defender',
                                                             'FW': 'Forward', 'GK': 'Goalkeeper'})

#             df = df.drop(columns=['Rk', 'Pos','Matches','Nation','Position_2','Tkl+Int',
#                                   'Def 3rd', 'Mid 3rd', 'Att 3rd','Att', 'Lost','Blocks','Position'])

            df.rename(columns={'Sh':'Blocks Sh','Pass':'Blocks Pass'}, inplace = True)
            defense[0][count] = df

            count += 1

    else:
        count = 0
        for df in shot[0]:

            # rename columns
            df = df.fillna(0)

            df['Age'] = df['Age'].str[:2]
            df['Position_2'] = df['Pos'].str[3:]
            df['Position'] = df['Pos'].str[:2]
            df['Nation'] = df['Nation'].str.split(' ').str.get(1)

            df['Position'] = df['Position'].replace({'MF': 'Midfielder', 'DF': 'Defender', 'FW': 'Forward', 'GK': 'Goalkeeper'})
            df['Position_2'] = df['Position_2'].replace({'MF': 'Midfielder', 'DF': 'Defender',
                                                             'FW': 'Forward', 'GK': 'Goalkeeper'})

#             df = df.drop(columns=['Rk', 'Pos','Matches','Nation','Position_2', 'Dist'])

            shot[0][count] = df

            count += 1

df_shot = pd.concat(shot[0])
df_pass = pd.concat(passing[0])
df_def = pd.concat(defense[0])

final = pd.merge(pd.merge(df_shot,df_def,on=['Player','Squad','Born','Based','Age','90s']),df_pass,on=['Player','Squad','Born','Based','Age','90s'])
final['player_code'] = final['Player'].str.lower().str.replace(' ','-')

replacements = {
    'à': 'a', 'á': 'a', 'â': 'a', 'ä': 'a', 'ǎ': 'a', 'æ': 'a', 'ã': 'a', 'å': 'a', 'ā': 'a',
    'è': 'e', 'é': 'e', 'ê': 'e', 'ë': 'e', 'ě': 'e', 'ẽ': 'e', 'ē': 'e', 'ė': 'e', 'ę': 'e',
    'ì': 'i', 'í': 'i', 'î': 'i', 'ï': 'i', 'ǐ': 'i', 'ĩ': 'i', 'ī': 'i', 'ı': 'i', 'į': 'i',
    'ò': 'o', 'ó': 'o', 'ô': 'o', 'ö': 'o', 'ǒ': 'o', 'œ': 'o', 'ø': 'o', 'õ': 'o', 'ō': 'o',
    'ù': 'u', 'ú': 'u', 'û': 'u', 'ü': 'u', 'ǔ': 'u', 'ũ': 'u', 'ū': 'u', 'ű': 'u', 'ů': 'u',
    'ğ':'g','ġ':'g','Ğ':'G','Ġ':'G','ç':'c','ć':'c','č':'c','ċ':'c','ł':'l','ļ':'l','ľ':'l',
    'ß':'s','ş':'s','ș':'s','ś':'s','š':'s','ķ':'k','ñ':'n','ń':'n','ņ':'n','ň':'n','ŵ':'w',
    'ź':'z','ž':'z','ż':'z','þ':'b',"'":'',"’":''
}

for key in replacements:
    final['player_code'] = final['player_code'].str.replace(key, replacements[key])

In [ ]:
final.to_csv('player_data_2023.csv')

In [ ]:
final['player_code']

In [ ]:
final[0]

In [ ]:
list(final['player_code'])